# Compare SECP256k1, SHA-256, RIPEMD160 and BASE58

In [2]:
import hashlib
import time
from ecdsa import SECP256k1, SigningKey
import binascii
import hashlib
import base58
import time
import os

values = [os.urandom(32) for _ in range(0, 10_000)]

## SECP256k1

In [3]:
start_time = time.time()

for value in values:
    result = SigningKey.from_string(value, curve=SECP256k1)
    result = result.get_verifying_key()
    result.to_string("compressed")

end_time = time.time()

print(f"Time taken (SECP256k1): {end_time - start_time:.2f} seconds")


Time taken (SECP256k1): 2.37 seconds


## SHA-256

In [7]:
start_time = time.time()

for value in values:
    result = hashlib.sha256(value).digest()
    result = hashlib.sha256(result).digest()
    result = hashlib.sha256(result).digest()

end_time = time.time()

print(f"Time taken (3 x SHA-256): {end_time - start_time:.2f} seconds")

Time taken (3 x SHA-256): 0.02 seconds


## RIPEMD160

In [8]:
start_time = time.time()

for value in values:
    result = hashlib.new('ripemd160', value).digest()

end_time = time.time()

print(f"Time taken (RIPEMD160): {end_time - start_time:.2f} seconds")

Time taken (RIPEMD160): 0.01 seconds


## BASE58

In [9]:
start_time = time.time()

for value in values:
    result = base58.b58encode(value).decode()

end_time = time.time()

print(f"Time taken (BASE58): {end_time - start_time:.2f} seconds")

Time taken (BASE58): 0.09 seconds


## All together

In [63]:
def private_to_public(private_key_bytes):
    private_key = SigningKey.from_string(private_key_bytes, curve=SECP256k1)
    public_key = private_key.get_verifying_key()
    return public_key.to_string("compressed")

def public_key_to_address(compressed_public_key_bytes):
    sha256_hash = hashlib.sha256(compressed_public_key_bytes).digest()
    ripemd160_hash = hashlib.new('ripemd160', sha256_hash).digest()
    versioned_payload = b'\x00' + ripemd160_hash
    checksum = hashlib.sha256(hashlib.sha256(versioned_payload).digest()).digest()[:4]
    payload_with_checksum = versioned_payload + checksum
    return base58.b58encode(payload_with_checksum).decode()

def generate_key(private_key):
    private_key_hex = '{:064x}'.format(private_key)
    private_key_bytes = binascii.unhexlify(private_key_hex)

    compressed_public_key = private_to_public(private_key_bytes)
    bitcoin_address = public_key_to_address(compressed_public_key)
    
    return (private_key_hex, binascii.hexlify(compressed_public_key).decode(), bitcoin_address)

start_time = time.time()

for value in values:
    generate_key(int.from_bytes(value, byteorder='big'))

end_time = time.time()

print(f"Time taken (all together): {end_time - start_time:.2f} seconds")

Time taken (all together): 4.82 seconds
